# Gather Customer Data from SQL Server

In [ ]:
import pymssql
import re
import pandas as pd
import pandas_dedupe
from ftfy import fix_text

#### Cleanup ugly strings that cause headaches for CSV reading

In [ ]:
dupsrx = re.compile(r'(.)\1{5,}') # Search for any thing that has more than 5 repeated characters
chars_to_remove = [")","(","|","[","]","{","}","'","\"", ","]
badcharsrx = re.compile('[' + re.escape(''.join(chars_to_remove)) + ']') # Replace quotes and commas.
badaddress = re.compile(r'bad address',flags=re.IGNORECASE)
unabletoforward = re.compile(r'unable to', flags=re.IGNORECASE)


def fixstring(string):
    string = str(string)
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    string = badcharsrx.sub('', string)
    string = string.replace('&', 'and')
    string = string.lower() # normalise case
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    if string == '' or string.isspace() or dupsrx.search(string) != None or badaddress.search(string) != None or unabletoforward.search(string) != None :
        string = 'nan'

    return string


#### Iterate through each database and download raw customer data.  Cleanup before outputting to CSV

In [ ]:
server = r'server' 
database = '' 
username = 'user' 
password = 'password' 
stores = [ '511', '512', '582', '310', '316', '321', '501', '504', '519', '550', '566' ]
custfile = open('customers.csv', 'w', encoding='utf-8')

custfile.write('ID,StoreID,Company,FirstName,LastName,Address,Address2,City,State,Zip,EmailAddress,PhoneNumber\n')
for stid in stores:
    print('Working on Store: ' + stid, end='')
    database = 's' + stid
    cnxn = pymssql.connect(server, username, password, database)
    cursor = cnxn.cursor()
    query = r"""select Customer.ID,'{storeid}' as StoreID,Company,FirstName,LastName,[Address],Address2,City,[State],Zip,EmailAddress,PhoneNumber
    from Customer
    where Customer.AccountNumber <> 'CASH' and 
        TotalVisits >= 3 and
        Customer.ID in (select distinct CustomerID from [Transaction])""".format(storeid=stid)
    cursor.execute(query)
    for row in cursor:
        ndx = 0
        strikes = 0
        outrow = ''
        for col in row:
            if ndx > 1:
                clean = fixstring(col)
            else:
                clean = str(col)
            if clean == 'nan':
                strikes = strikes+1
            outrow = outrow + clean
            ndx = ndx + 1 
            if ndx < len(row):
                outrow = outrow + ','               
                
        if strikes <= 4:
            custfile.write(outrow)
            custfile.write('\n')
    print(' Done.')

custfile.close()

#### Attempt Training on subset of actual file.

In [ ]:
df = pd.read_csv('customers.csv', encoding='utf-8', nrows=20000)
df = pandas_dedupe.dedupe_dataframe(df, ['Company', 'FirstName', 'LastName', 'Address', 'City', 'State', 'Zip', 'PhoneNumber'])

#### Re-Read entire file and dedupe based on training.

In [18]:
df = pd.read_csv('customers.csv', encoding='utf-8', nrows=140000)
df = pandas_dedupe.dedupe_dataframe(df, ['Company', 'FirstName', 'LastName', 'Address', 'City', 'State', 'Zip', 'PhoneNumber'])

Importing data ...


C:\Users\anand.HOME\AppData\Local\Temp\ipykernel_9596\61623702.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df = pandas_dedupe.dedupe_dataframe(df, ['Company', 'FirstName', 'LastName', 'Address', 'City', 'State', 'Zip', 'PhoneNumber'])


Reading from dedupe_dataframe_learned_settings
Clustering...


A component contained 66138 elements. Components larger than 30000 are re-filtered. The threshold for this filtering is 2.735057171410285e-24
A component contained 66137 elements. Components larger than 30000 are re-filtered. The threshold for this filtering is 7.55496214688561e-24
A component contained 66114 elements. Components larger than 30000 are re-filtered. The threshold for this filtering is 2.0549916930140488e-23
A component contained 66088 elements. Components larger than 30000 are re-filtered. The threshold for this filtering is 5.587602355957443e-23
A component contained 65934 elements. Components larger than 30000 are re-filtered. The threshold for this filtering is 1.5189199419679814e-22
A component contained 65460 elements. Components larger than 30000 are re-filtered. The threshold for this filtering is 4.1288524771356315e-22
A component contained 64761 elements. Components larger than 30000 are re-filtered. The threshold for this filtering is 1.122342747488234e-21
A co

# duplicate sets 129880


### Get Transactions for customers:  
Cluster Customers to a sigle ID; Label based on price category, department and brand

In [ ]:
df.to_csv('scrubbed.csv')
grouped = df.groupby("StoreID")

In [ ]:
import pymssql
tranfile = open('transactions.csv', 'w', encoding='utf-8')
tranfile.write('CustomerID,ItemLookupCode,TranTime,PriceCategory,Department,Brand\n')

def priceCategory(price):
    if price < 10:
        return '1'
    if price >=10 and price < 100:
        return '2'
    if price >=100 and price < 500:
        return '3'
    if price >=500 and price < 1000:
        return '4'
    if price >=1000:
        return '5'
    return '0'

for name, group in grouped:
    print('Working on Store: ' + name)
    database = 's' + name
    cnxn = pymssql.connect(server, username, password, database)
    cursor = cnxn.cursor()
    for index, entry in df.iterrows():
        query = r"""select Customer.ID,ItemLookupCode, [Transaction]."Time", TransactionEntry.Price, TransactionEntry.Quantity, Department.Name, BinLocationXRef.RetailCD
from Customer, [Transaction], TransactionEntry, Item, Department, BinLocationXRef
where Customer.ID=[Transaction].CustomerID and 
      [Transaction].TransactionNumber=TransactionEntry.TransactionNumber and
      Item.ID=TransactionEntry.ItemID and 
      Department.ID=Item.DepartmentID and
      item.BinLocation=BinLocationXRef.RetailBinCD and
      Customer.ID={custid} and Department.Code not in ('WP01', '990', '970', 'GC','TAX', 'SHIP' )""".format(custid=entry['ID'])
        cursor.execute(query)
        for row in cursor:
            tranfile.write(entry[0] + ',' + str(row[1]) + ',' + str(row[2]) + ',' + priceCategory(row[3]) + ',' + str(row[5]) + ',' + str(row[6]) +'\n')
    print('Done!')
tranfile.close()